In [8]:
import math
import pygal
import time
import sys

In [9]:
# Fibonacci with 3 implementation, brute-force, dynamic programming, matrix 

In [10]:
def fibonacci_brute_force(n):
    if n == 0 or n == 1:
        return n
    else:
        return fibonacci_brute_force(n - 1) + fibonacci_brute_force(n - 2)

In [11]:
def fibonacci_memorize(n, dict_ = None):
    if dict_ == None:
        dict_ = {}
    if n == 0 or n == 1:
        return n
    if n in dict_:
        return dict_[n]
    else:
        ans = fibonacci_memorize(n - 1, dict_) + fibonacci_memorize(n - 2, dict_)
        dict_[n] = ans
        return ans

In [12]:
d = {}
test = fibonacci_memorize(60, d)